# 컨텐츠 기반 필터링(Content-based filtering)

In [12]:
import numpy as np
import pandas as pd

In [13]:
from surprise import Dataset
data = Dataset.load_builtin('ml-100k', prompt=False)
df = pd.DataFrame(data.raw_ratings, columns=['user-id','movie-id','rating','timestamp'])    
df.head()

,user-id,movie-id,rating,timestamp
0,196,242,3.0,881250949
1,186,302,3.0,891717742
2,22,377,1.0,878887116
3,244,51,2.0,880606923
4,166,346,1.0,886397596


In [14]:
df.shape

(100000, 4)

특정 아이디(rows)의 사람이 특정 영화(columns)의 평점을 몇 점을 매겼는지 DataFrame으로 만들어서 분석하기 

- Adjacent Matrix 생성
    - 행(row) == 사용자
    - 열(column) == 영화
    - 내용 == 평점

In [15]:
# Python 인덱스는 맨 첫번째가 0으로 시작, ID는 1부터 시작함
raw_data = np.array(data.raw_ratings, dtype=int)
raw_data[:, :2] -= 1            # user-id, movie-id가 0부터 시작하도록 재설정
raw_data[:5]

array([[      195,       241,         3, 881250949],
       [      185,       301,         3, 891717742],
       [       21,       376,         1, 878887116],
       [      243,        50,         2, 880606923],
       [      165,       345,         1, 886397596]])

In [22]:
n_users = df['user-id'].nunique()
n_movies = df['movie-id'].astype(int).max()
n_users, n_movies

(943, 1682)

- adj_matrix: 0/1, 평점점수를 주는 경우

In [24]:
# 본 영화는 1, 안 본 영화는 0
adj_matrix = np.zeros((n_users, n_movies), int)
for user_id, movie_id, _, _ in raw_data:
    adj_matrix[user_id, movie_id] = 1      
adj_matrix[:5]

array([[1, 1, 1, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0]])

In [27]:
# 유사도 - 이진 벡터의 내적
my_id, my_vector = 0, adj_matrix[0]

np.dot(my_vector, adj_matrix[1]), np.dot(my_vector, adj_matrix[2])

(18, 8)

In [30]:
# 누가 나(user-id가 0)랑 가장 닮았는가?
best_score, best_match_id = -1, -1

for i in range(1, len(adj_matrix)):
    dot = np.dot(my_vector, adj_matrix[i])
    if dot > best_score:
        best_score = dot
        best_match_id = i

best_score, best_match_id

(183, 275)